In [1]:
import pandas as pd
import numpy as np
from lightgbm.sklearn import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.compose import make_column_transformer

In [2]:
DIR = "../data/"
SMOOTHIE = "Smoothie King/smoothie_king_"

In [3]:
smoothie_demographic = pd.read_csv(DIR + SMOOTHIE + "demographic_variables.csv")
smoothie_stores = pd.read_csv(DIR + SMOOTHIE + "stores.csv")
smoothie_poi_variables = pd.read_csv(DIR + SMOOTHIE + "poi_variables.csv")
smoothie_demographic

,store,medhhinc_1mi,age0018_1mi,age0018_p_1mi,age65pl_1mi,age65pl_p_1mi,age85pl_1mi,age85pl_p_1mi,asian_1mi,asian_p_1mi,...,poverty_inpoverty_10mi,poverty_inpoverty_p_10mi,spend_breakfastbrunch_10mi,spend_dinner_10mi,spend_foodbev_10mi,spend_lunch_10mi,wealth_hhavg_10mi,wealth_hhtotal_10mi,white_10mi,white_p_10mi
0,SK 1504,83846.0042,3774.0,0.2969,1151.0,0.0905,57.0,0.0045,603.0579,0.0474,...,15484.0,0.0640,62995242.0,364526249.0,2.039742e+09,205361587.0,244674.0,241941.0,332706.8340,0.5118
1,SK 0057,50388.0216,1688.0,0.2298,1311.0,0.1784,123.0,0.0168,280.1880,0.0381,...,21823.0,0.1813,27814428.0,149355911.0,9.420898e+08,86239896.0,221080.0,120338.0,160816.1504,0.5205
2,SK 1415,59999.1012,3340.0,0.2405,2451.0,0.1765,194.0,0.0140,167.5000,0.0121,...,10340.0,0.0608,41122434.0,231354667.0,1.394461e+09,131310533.0,252962.0,169964.0,343932.2978,0.8088
3,SK 1231,97438.8015,2266.0,0.1938,2476.0,0.2118,224.0,0.0192,414.8301,0.0355,...,59470.0,0.1408,102293013.0,554039317.0,3.620776e+09,317547521.0,230357.0,422361.0,400703.7973,0.3940
4,SK 1535,56156.2657,9780.0,0.3207,2628.0,0.0862,91.0,0.0030,2148.1145,0.0705,...,37120.0,0.0775,115249938.0,662800687.0,4.112360e+09,370573539.0,245089.0,479049.0,580244.6476,0.4906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
791,SK 1536,32998.8647,3424.0,0.3104,1395.0,0.1265,98.0,0.0089,79.8703,0.0072,...,37636.0,0.1624,66296019.0,359428431.0,2.322572e+09,207584850.0,212492.0,231740.0,116424.0527,0.1615
792,SK 1886,55847.7090,2590.0,0.1828,1542.0,0.1089,229.0,0.0162,556.5048,0.0393,...,12511.0,0.1215,23624568.0,134117651.0,9.156644e+08,76271730.0,248551.0,102970.0,214301.5774,0.8203
793,SK 0162,51203.5596,1878.0,0.2856,1024.0,0.1558,126.0,0.0191,84.8653,0.0129,...,9097.0,0.0881,25916308.0,146563993.0,9.092607e+08,82836656.0,249492.0,103313.0,192989.8084,0.6788
794,SK 1449,50153.8368,2000.0,0.2824,847.0,0.1196,84.0,0.0118,181.6803,0.0257,...,21515.0,0.1795,30289183.0,161335026.0,1.031824e+09,93613567.0,210307.0,119859.0,129891.1147,0.4147


In [4]:
smoothie_poi_variables

,store,centerxy_gla_1mi,centerxy_count_1mi,centerxy_gla_2mi,centerxy_count_2mi,centerxy_gla_3mi,centerxy_count_3mi,centerxy_gla_5mi,centerxy_count_5mi,centerxy_gla_10mi,...,nces_public_schools_total_enrollment_10mi,nces_public_schools_10mi,osm_nearest_exit_dist,osm_highway_exits_count_1mi,osm_highway_exits_count_2mi,osm_highway_exits_count_3mi,osm_highway_exits_count_5mi,osm_highway_exits_count_10mi,transitstop_nearest_dist,transitstops
0,SK 1504,198023.0,1,307544.0,2,1969648.0,11,5770569.0,20,15317967.0,...,110757.0,174,1.875541,0,2,17,45,123,14.818824,0
1,SK 0057,253302.0,2,1002488.0,7,3381962.0,11,5522926.0,20,6507331.0,...,45635.0,78,2.617072,0,0,3,14,58,49.572856,0
2,SK 1415,715612.0,3,1361502.0,7,2027580.0,10,4001581.0,16,13191004.0,...,55532.0,96,0.194937,2,8,25,45,162,3.518308,0
3,SK 1231,415300.0,2,588410.0,3,1850924.0,5,4825186.0,14,21089283.0,...,121834.0,184,1.393043,0,6,31,61,114,0.021790,25
4,SK 1535,308913.0,2,1148114.0,7,3823444.0,13,10583744.0,45,36941461.0,...,179702.0,280,0.711949,3,14,23,72,366,0.055289,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
791,SK 1536,1333256.0,5,1921864.0,9,2043636.0,10,3504457.0,16,8561361.0,...,136369.0,254,0.261721,14,23,36,81,258,0.012345,24
792,SK 1886,367642.0,2,1734971.0,7,2749951.0,13,4156741.0,17,4538137.0,...,36249.0,67,4.012518,0,0,0,10,17,0.189059,13
793,SK 0162,628704.0,3,1294244.0,7,2641385.0,11,2922800.0,12,4807504.0,...,48840.0,61,0.397305,4,11,15,23,38,0.699036,0
794,SK 1449,3191061.0,8,3463831.0,10,4154627.0,13,4450657.0,15,5950658.0,...,41121.0,75,0.706073,5,7,12,46,109,4.771075,0


In [5]:
smoothie_merged = smoothie_stores.merge(smoothie_demographic).merge(smoothie_poi_variables)
smoothie_merged

,store,longitude,latitude,category,cbsa_name,dma_name,state_name,market_size,store_density,medhhinc_1mi,...,nces_public_schools_total_enrollment_10mi,nces_public_schools_10mi,osm_nearest_exit_dist,osm_highway_exits_count_1mi,osm_highway_exits_count_2mi,osm_highway_exits_count_3mi,osm_highway_exits_count_5mi,osm_highway_exits_count_10mi,transitstop_nearest_dist,transitstops
0,SK 1504,-97.650392,30.519353,SHOPPING,"Austin-Round Rock, TX","Austin, TX",Texas,Large Metro (2),Light Suburban,83846.0042,...,110757.0,174,1.875541,0,2,17,45,123,14.818824,0
1,SK 0057,-88.171150,30.672501,SHOPPING,"Mobile, AL","Mobile et al, AL-FL",Alabama,Medium City (4),Light Suburban,50388.0216,...,45635.0,78,2.617072,0,0,3,14,58,49.572856,0
2,SK 1415,-90.535722,38.784250,HOME,"St. Louis, MO-IL","St. Louis, MO",Missouri,Very Large Metro (1),Light Suburban,59999.1012,...,55532.0,96,0.194937,2,8,25,45,162,3.518308,0
3,SK 1231,-80.134700,26.100737,TRAVEL,"Miami-Fort Lauderdale et al, FL","Miami-Ft. Lauderdale, FL",Florida,Very Large Metro (1),Suburban,97438.8015,...,121834.0,184,1.393043,0,6,31,61,114,0.021790,25
4,SK 1535,-96.856651,32.996408,WORK,"Dallas-Fort Worth-Arlington, TX","Dallas-Ft. Worth, TX",Texas,Very Large Metro (1),Light Suburban,56156.2657,...,179702.0,280,0.711949,3,14,23,72,366,0.055289,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
791,SK 1536,-96.872596,32.647809,HOME,"Dallas-Fort Worth-Arlington, TX","Dallas-Ft. Worth, TX",Texas,Very Large Metro (1),Exurban,32998.8647,...,136369.0,254,0.261721,14,23,36,81,258,0.012345,24
792,SK 1886,-105.077634,40.564695,OTHER,"Fort Collins, CO","Denver, CO",Colorado,Medium City (4),Light Suburban,55847.7090,...,36249.0,67,4.012518,0,0,0,10,17,0.189059,13
793,SK 0162,-95.478001,30.316531,SHOPPING,"Houston-The Woodlands et al, TX","Houston, TX",Texas,Very Large Metro (1),Exurban,51203.5596,...,48840.0,61,0.397305,4,11,15,23,38,0.699036,0
794,SK 1449,-78.968258,35.064994,SHOPPING,"Fayetteville, NC","Raleigh et al, NC",North Carolina,Medium City (4),Exurban,50153.8368,...,41121.0,75,0.706073,5,7,12,46,109,4.771075,0


In [6]:
smoothie_merged = smoothie_merged.drop(columns=["store", "longitude", "latitude", "cbsa_name"])
smoothie_merged

,category,dma_name,state_name,market_size,store_density,medhhinc_1mi,age0018_1mi,age0018_p_1mi,age65pl_1mi,age65pl_p_1mi,...,nces_public_schools_total_enrollment_10mi,nces_public_schools_10mi,osm_nearest_exit_dist,osm_highway_exits_count_1mi,osm_highway_exits_count_2mi,osm_highway_exits_count_3mi,osm_highway_exits_count_5mi,osm_highway_exits_count_10mi,transitstop_nearest_dist,transitstops
0,SHOPPING,"Austin, TX",Texas,Large Metro (2),Light Suburban,83846.0042,3774.0,0.2969,1151.0,0.0905,...,110757.0,174,1.875541,0,2,17,45,123,14.818824,0
1,SHOPPING,"Mobile et al, AL-FL",Alabama,Medium City (4),Light Suburban,50388.0216,1688.0,0.2298,1311.0,0.1784,...,45635.0,78,2.617072,0,0,3,14,58,49.572856,0
2,HOME,"St. Louis, MO",Missouri,Very Large Metro (1),Light Suburban,59999.1012,3340.0,0.2405,2451.0,0.1765,...,55532.0,96,0.194937,2,8,25,45,162,3.518308,0
3,TRAVEL,"Miami-Ft. Lauderdale, FL",Florida,Very Large Metro (1),Suburban,97438.8015,2266.0,0.1938,2476.0,0.2118,...,121834.0,184,1.393043,0,6,31,61,114,0.021790,25
4,WORK,"Dallas-Ft. Worth, TX",Texas,Very Large Metro (1),Light Suburban,56156.2657,9780.0,0.3207,2628.0,0.0862,...,179702.0,280,0.711949,3,14,23,72,366,0.055289,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
791,HOME,"Dallas-Ft. Worth, TX",Texas,Very Large Metro (1),Exurban,32998.8647,3424.0,0.3104,1395.0,0.1265,...,136369.0,254,0.261721,14,23,36,81,258,0.012345,24
792,OTHER,"Denver, CO",Colorado,Medium City (4),Light Suburban,55847.7090,2590.0,0.1828,1542.0,0.1089,...,36249.0,67,4.012518,0,0,0,10,17,0.189059,13
793,SHOPPING,"Houston, TX",Texas,Very Large Metro (1),Exurban,51203.5596,1878.0,0.2856,1024.0,0.1558,...,48840.0,61,0.397305,4,11,15,23,38,0.699036,0
794,SHOPPING,"Raleigh et al, NC",North Carolina,Medium City (4),Exurban,50153.8368,2000.0,0.2824,847.0,0.1196,...,41121.0,75,0.706073,5,7,12,46,109,4.771075,0


In [7]:
train_df, test_df = train_test_split(smoothie_merged, test_size=0.2, random_state=42)
X_train = train_df.drop(columns=["category"])
y_train = train_df["category"]
X_test = test_df.drop(columns=["category"])
y_test = test_df["category"]

In [8]:
X_train["store_density"].unique()

array(['Light Suburban', 'Exurban', 'Rural', 'Suburban', 'Light Urban',
       'Urban', 'Super Urban'], dtype=object)

In [9]:
categorical_features = ["dma_name", "store_density", "state_name"]
ordinal_features = ["market_size"]
numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()

In [10]:
market_levels = [
    "Small Town (6)",
    "Small City (5)",
    "Medium City (4)",
    "Large City (3)",
    "Large Metro (2)",
    "Very Large Metro (1)"
]

ordinal_transformer = OrdinalEncoder(categories=[market_levels], dtype=int)
    
preprocessor = make_column_transformer(
    (StandardScaler(), numeric_features),
    (ordinal_transformer, ordinal_features),
    (OneHotEncoder(handle_unknown="ignore", sparse=False), categorical_features)
)
pipe = make_pipeline(
    preprocessor,
    LGBMClassifier(random_state=42)
)
pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['medhhinc_1mi',
                                                   'age0018_1mi',
                                                   'age0018_p_1mi',
                                                   'age65pl_1mi',
                                                   'age65pl_p_1mi',
                                                   'age85pl_1mi',
                                                   'age85pl_p_1mi', 'asian_1mi',
                                                   'asian_p_1mi',
                                                   'avg_faminc_1mi',
                                                   'avghhinc_1mi', 'black_1mi',
                                                   'black_p_1mi', 'boomer_1mi',
                                                   'boomer_p_1mi',
                                                   'com0002_p_1mi',
                                                   'com0205_p_1...
                                                  OrdinalEncoder(categories=[['Small '
                                                                              'Town '
                                                                              '(6)',
                                                                              'Small '
                                                                              'City '
                                                                              '(5)',
                                                                              'Medium '
                                                                              'City '
                                                                              '(4)',
                                                                              'Large '
                                                                              'City '
                                                                              '(3)',
                                                                              'Large '
                                                                              'Metro '
                                                                              '(2)',
                                                                              'Very '
                                                                              'Large '
                                                                              'Metro '
                                                                              '(1)']],
                                                                 dtype=<class 'int'>),
                                                  ['market_size']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['dma_name', 'store_density',
                                                   'state_name'])])),
                ('lgbmclassifier', LGBMClassifier(random_state=42))])

In [11]:
pipe.score(X_test, y_test)

0.44375